In [1]:
#only doing vertical spreads, not single calls,puts
#no conditionals yet

import json
import datetime
import inspect
import re
from mt_classes import *
from mt_tokens import *
import unidecode
from IPython.display import display, Markdown, Latex

#PROCESS_ONLY_SYM = "PYPL"
PROCESS_ONLY_SYM = None



In [2]:
#with open("swing_1_9_22_20.json","r") as read_file:
#    data = json.load(read_file)
with open("cl_weekly_1_9_22_20.json","r") as read_file:
    data = json.load(read_file)
    
    

In [4]:

GREEN = '<div class="alert-success">'
YELLOW = '<div class="alert-warning">'
RED = '<div class="alert-danger">'
BLUE = '<div class="alert-info">'
END = '</div>'



EVENT_SELLING = 0

    
class Adj():
    pass
    
m = []

def seq_push(pos,letter,obj):
    global s,o
    
    s = s + letter
    o.append(obj)
    
    obj.index = pos
    
def filter(symb,text):
    
    text_split = text.split()
    
    for i,w in enumerate(text_split):
        
        if det_symbol(i,w):
            token = Symbol(i,w)

            return token.sym == symb
            
    return False


def has_class(tokens,cl):
    for token in tokens:
        if type(token) is cl:
            return True
    return False

def get_token(tokens,cl):
    for token in tokens:
        if type(token) is cl:
            return token
        
    return None

def get_token_after(tokens,cl1,cl2):
    
    found = False
    for token in tokens:
        if found is False:
            if inspect.isclass(cl1):
                if type(token) is cl1:
                    found = True
            else:
                if token is cl1:
                    found = True
        else:
            if type(token) is cl2:
                return token
    return None
    

count = 0


def count_shorted(tokens,timestamp,text):
    
    shorted1 = get_token(tokens,Action_shorting)
    shorted2 = get_token_after(tokens,shorted1,Action_shorting)
    
    if shorted1 and shorted2:
        print(text)
    pass

def get_stop(tokens,timestamp,text,options):

    text_split = text.split()

    symbol = get_token(tokens,Symbol)
    
    if not symbol:
        print('get stop: no symbol',text)
        return options
    
    stop = get_token(tokens,Stop)
    
    
    if stop:
        #print("get_stop: found stop at:",stop.index,len(text_split))
        stop_fields = {}
        stop_fields['ts'] = timestamp
        stop_fields['sym'] = '{}'.format(symbol)
        if len(text_split)-1 > stop.index:
            #print("get_stop more words")
            next = text_split[stop.index + 1]
            next = next.rstrip(',.')
            price = Price(0,next)
            if price.valid:
                stop_fields['type'] = 'price'
                stop_fields['price'] = price.price
                #print(symbol,'stop1:',price,next)
            else:
                if get_token_after(tokens,stop,Premium):
                    stop_fields['type'] = '2xpremium'
                else:
                    if next.lower() == 'breakeven':
                        #print('get stop breakeven')
                        stop_fields['type'] = 'breakeven'
                    else:
                        if next.lower() == 'tighter':
                            price = get_token_after(tokens,stop,Price)
                            if price:
                                stop_fields['type'] = 'tighter'
                                stop_fields['price'] = price.price
                        else:
                            if next.lower() == 'low':
                                stop_fields['type'] = 'low'
                            else:
                                if next.lower() == 'high':
                                    stop_fields['type'] = 'high'
                                else:
                                    print(symbol,'stop word:[',next,']',text)
                
        if len(stop_fields) > 2:
            if len(options):
                options[-1]['stop']= stop_fields
                
            else:
                print('here is a naket stop')
                options.append({'ts': timestamp,'stop': stop_fields})

            pass # no option so have to create stop entry
    else:
        #print(symbol,"no stop")
        pass
        
        
    return options

    

def get_options(tokens,timestamp,text):
    global count
    
    
    options = []
    
    action = None
    symbol = None
    condition = None
    
#    print("get option entry: ",text)
    
    for i,token in enumerate(tokens):
        
        if not symbol:
            if type(token) is Symbol:
                symbol = token

        if action:
            
            if i+1 < len(tokens):
                next_token = tokens[i+1]
            else:
                next_token = None
            
            if type(token) is Strike_date or type(token) is Strike_day:
                strike_date = token
                if type(next_token) is Spread_type:
                    spread_type = next_token
                
            if type(token) is Strikes:
                strikes = token
                if type(next_token) is Spread_type:
                    spread_type  = next_token
                    

                
            #if symbol and spread_type and strike_date and strikes:
            if symbol and strike_date and strikes:  # if no spread_type, then deduce from strikes and current price

                #if type(action) is Action_shorting:
                count = count + 1
                #print("get options, found option",count,symbol,spread_type,strike_date,strikes)
                option = {}
                option['ts'] = timestamp
                option['action'] = str(action)
                option['sym'] = '{}'.format(symbol)
                option['spread_type'] = str(spread_type)
                option['strike_date'] = '{}'.format(str(strike_date))
                option['strikes'] = strikes.aslist()
                if condition:
                    option['condition'] = True

                options.append(option)
                
                action = None

        else:
            
            if not condition:
                if type(token) is Condition:

                    condition = token;
                
                
            if type(token) is Action_shorting:
                action = token
            if type(token) is Action_shorted:
                action = token
                
#            print('action',str(action))
                
            spread_type = strike_date = strikes = None
            
    if len(options):
        pass
    else:
#        mt = GREEN + 'no options foune: ' + str(symbol) + ' ' + text + END
#        display(Markdown(mt))
        print("no options found for [{0}] text:".format(symbol),text)
        pass
    
            
    return options

def check_pair_dte(timestamp,tokens,i,token1,token2):
    
    if token2.text.upper().startswith('DTE'):
        strike_date = Strike_date(i,token2.text,timestamp,token1.text)
        if strike_date.valid:
            tokens[i].valid = False
            tokens[i+1] = strike_date
            return True
    
    return False

def check_pair_date_strikes(timestamp,tokens,i,token1,token2):
    
    if type(token2) is Unknown:
        if type(token1) is Unknown:
            
            pair1 = token1.text.split('/')
            pair2 = token2.text.split('/')
            
            if len(pair1) == 2 and len(pair2) ==2:
                if pair1[0].startswith('(') and pair1[1].endswith(')'):
                
                    sd = Strike_date(i,token1.text,timestamp,"")
                    strikes = Strikes(i,token2.text)
                    if sd.valid and strikes.valid:
                        tokens[i] = sd
                        tokens[i+1] = strikes
                        return True
        else:
            if type(token1) is Strike_date:
                strikes = Strikes(i,token2.text)
                if strikes.valid:
                    tokens[i+1] = strikes
                    return True
    return False

    

def check_pair_lot_quantity(timestamp,tokens,i,token1,token2):
    
    if type(token1) is Unknown and type(token2) is Unknown:
            
        lotQuantity = LotQuantity(i,token1.text,token2.text)
        if lotQuantity.valid:
            token1.valid = False
            tokens[i+1] = lotQuantity
            return True
        
    return False

            
            
def analyse_pairs(timestamp,tokens,function):
    
    for i in range(len(tokens)-1):
        token1 = tokens[i]
        token2 = tokens[i+1]
        
        if function(timestamp,tokens,i,token1,token2):
            break
            
    tokens = [token for token in tokens if token.valid] #remove any invalid tokens
    
    return tokens
        
        

def create_tokens2(timestamp,text):
    
    
    tokens = [Unknown(pos,fragment) for pos,fragment in enumerate(text.split())]
    

    tokens = analyse_pairs(timestamp,tokens,check_pair_dte)
    
    tokens = analyse_pairs(timestamp,tokens,check_pair_lot_quantity)
    
    tokens = analyse_pairs(timestamp,tokens,check_pair_date_strikes)
                                        
    tokens = create_tokens(timestamp,tokens)
    
            
    tokens = [token for token in tokens if type(token) != Unknown]
    
    return tokens
    
    

def process_text(timestamp,text):
    global count
    
    
    
    if PROCESS_ONLY_SYM:
        if filter(PROCESS_ONLY_SYM,text) is False:
            return []
    
    tokens = create_tokens2(timestamp,text)
    

    
#    mt = YELLOW + 'tokens:' + str(tokens) + END
#    mt = '<div>' + 'tokens:' + str(tokens) + END
    #display(Markdown(mt))

#<div class="alert-warning">this is a green</div>
    #print()
    
    #print('text',text)
    #print('tokens:',tokens)

    
    
    #get_stop(timestamp,text)
    
    options = get_options(tokens,timestamp,text)
    
    
    #options = get_stop(tokens,timestamp,text,options)
    
    #print("{} options: {} {}".format(timestamp,count,options))
    
#    display(Markdown(options))

#    print(options)
    
    #count_shorted(timestamp,text); return
    
#    if options is None:
#        print('process_text none',text)

    #print()

    return options

        

    
def process_element(timestamp,element):

        
    if isinstance(element,list):
        
        #remove the embedded dictionaries and concat the strings
        
        complete = ''
        
        for l in element:
            if isinstance(l,str) is not True:
                continue
                
            complete = complete + l
            
        element = complete
        
    if isinstance(element,str):
        element = unidecode.unidecode(element)  # convert the occasional unicode character like '–' to ascii '-'
        options = process_text(timestamp,element)
        return options
    
    return []

print('channel id:',data['id'])

options = []

for message in data['messages']:
#    if message['type'] != 'message':
#        continue

#    print(message)
    
    if 'text' in message.keys():
        options.extend(process_element(message['date'],message['text']))


#    try:
#        text = message['text']
#        process_element('date',text)
#    except:
#        print('failure:',message)
#        break

print()
print()
display(Markdown('<div class="alert-success">...</div>'))

for option in options:
    #print()
    #print("{}:option: {}".format(option['ts'],option))
    
    if "action" in option:

        if type(option['action']) is Action_shorting:
            if 'stop' in option:
                #print(option['sym'],"option has stop")
                pass
            else:
                #print(option['sym'],"option has no stop")
                pass
    else:
        if 'stop' in option:
                #print(option['stop']['sym'],"Naket Stop")
                pass

with open('actions.json','w') as outfile:
    json.dump(options,outfile)


channel id: 10015910801
no options found for [SPX] text: 5/27:  Shorting May (5/27) SPX 2950/2925 put spread for $.75/share, will advise on stop
no options found for [SPX] text: move stop breakeven on SPX put spread
no options found for [None] text: cover here $.15/share, nice trade
no options found for [None] text: Move stop breakeven when you are halfway to max profit
no options found for [HES] text: HES - stop 5 min high on call spread
no options found for [HES] text: HES - 6/2:  Roll the short Jun (6/5) $46 puts to $50 puts for $.50/share to increase total premium to $1.33/share, new stop $49.72
no options found for [GOOGL] text: GOOGL - shorting Jun 5th 1400/1390 put spread for $.90/share
no options found for [BA] text: BA - adding another put spread:  Jun (6/12) $144/134 for around $.75/share to increase total premium.  New stop for both $144.48
no options found for [GOOGL] text: GOOGL - move stop breakeven on put spread, market feeling toppy here
no options found for [COST] text

<div class="alert-success">...</div>

<div class="alert-warning">this is a green</div>

<div class="alert-danger">this is a green</div>

foo <font color='red'>bar</font> foo

In [ ]:
print(options[5])

In [ ]:
if 'action' in options[5]:
    print('abc')

In [ ]:
{'timestamp': '2020-05-26T06:52:24', 'action': 'shorting', 'sym': 'ZM', 'spread_type': 'CALL', 'strike_date': '2020-05-29', 'strikes': [180.0, 185.0], 'stop': {'timestamp': '2020-05-26T06:52:24', 'sym': 'ZM', 'type': 'price', 'price': 176.22}}